### Importing libraries

In [2]:
import chess
import numpy as np

### Global Variables

In [3]:
board = chess.Board()

### Function for board to bitboard (upload this to mongo)

In [4]:
def board_to_bitboards(board):
    bitboards = {}
    for color in (chess.WHITE, chess.BLACK):
        for piece_type in chess.PIECE_TYPES:
            bitboard = 0
            for square in chess.SQUARES:
                piece = board.piece_at(square)
                if piece and piece.piece_type == piece_type and piece.color == color:
                    bitboard |= 1 << square
            color_name = 'White' if color == chess.WHITE else 'Black'
            piece_name = chess.piece_name(piece_type).capitalize()
            bitboards[f"{color_name} {piece_name}"] = bitboard
    return bitboards


In [5]:
board_to_bitboards(board)

{'White Pawn': 65280,
 'White Knight': 66,
 'White Bishop': 36,
 'White Rook': 129,
 'White Queen': 8,
 'White King': 16,
 'Black Pawn': 71776119061217280,
 'Black Knight': 4755801206503243776,
 'Black Bishop': 2594073385365405696,
 'Black Rook': 9295429630892703744,
 'Black Queen': 576460752303423488,
 'Black King': 1152921504606846976}

### Converting Bitboards to binary

In [6]:
bitboards = board_to_bitboards(board)
for name, bitboard in bitboards.items():
    print(f"{name} Bitboard: {bitboard:064b}")


White Pawn Bitboard: 0000000000000000000000000000000000000000000000001111111100000000
White Knight Bitboard: 0000000000000000000000000000000000000000000000000000000001000010
White Bishop Bitboard: 0000000000000000000000000000000000000000000000000000000000100100
White Rook Bitboard: 0000000000000000000000000000000000000000000000000000000010000001
White Queen Bitboard: 0000000000000000000000000000000000000000000000000000000000001000
White King Bitboard: 0000000000000000000000000000000000000000000000000000000000010000
Black Pawn Bitboard: 0000000011111111000000000000000000000000000000000000000000000000
Black Knight Bitboard: 0100001000000000000000000000000000000000000000000000000000000000
Black Bishop Bitboard: 0010010000000000000000000000000000000000000000000000000000000000
Black Rook Bitboard: 1000000100000000000000000000000000000000000000000000000000000000
Black Queen Bitboard: 0000100000000000000000000000000000000000000000000000000000000000
Black King Bitboard: 00010000000000000000000

### Full function for reshaping bitboards from mongo

In [7]:
def bitboard_to_matrix(bitboard):
    return np.array([(bitboard >> shift) & 1 for shift in range(64)]).reshape(8, 8)
def create_cnn_input(bitboards):
    layers = []
    for key in sorted(bitboards.keys()):  # Ensure consistent order
        matrix = bitboard_to_matrix(bitboards[key])
        print(matrix)
        layers.append(matrix)
    return np.stack(layers, axis=-1)
create_cnn_input(bitboards)

[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 1 0 0 1 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 1 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]]
[[0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 1]]
[[0 0 1 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]],

       [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 

In [ ]:
def process_sqlite_boards_to_mongo(self, batch_size: int = 5):
    with tf.io.TFRecordWriter(Settings().recordsData) as writer:
        with SessionLocal() as db:
            row_count = get_rollup_row_count(db=db)
            batch = fetch_all_game_positions_rollup(yield_size=500, db=db)
            serialized_examples = []  # List to accumulate serialized examples
            for game in tqdm(batch, total=row_count, desc="Processing Feature Data"):
                try:
                    if game:
                        self.evaluator.setup_parameters_gamepositions(game=game)
                        score = self.evaluator.get_board_scores_records()

                        serialized_data = serialize_data(score)
                        
                        features = {
                        'bitboards': _bytes_feature(serialized_data[0].numpy()),
                        'metadata': _bytes_feature(serialized_data[1].numpy()),
                        'target': _bytes_feature(serialized_data[2].numpy())
                        }
                        
                        serialized_data = tf.train.Example(features=tf.train.Features(feature=features))

                        serialized_examples.append(serialized_data)

                        # Check if we've accumulated enough examples to write a batch
                        if len(serialized_examples) >= batch_size:
                            for serialized_example in serialized_examples:
                                writer.write(serialized_example.SerializeToString())
                            serialized_examples = []  # Reset the list after writing
                    else:
                        return 1
                except Exception as e:
                    raise Exception(e)
            
            # Write any remaining examples after looping through all games
            for serialized_example in serialized_examples:
                writer.write(serialized_example)